In [14]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
import cufflinks as cf
import pathlib
import pandas_profiling
import statsmodels.api as sm

In [2]:
#We set the all charts as public
cf.set_config_file(sharing='public',theme='pearl',offline=False)
cf.go_offline()

# Regression

In [128]:
cwd = pathlib.Path.cwd()

data = pd.read_csv(cwd/'mpg_dataset'/'auto-mpg.csv')

data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [129]:
report = data.profile_report()
report.to_file(output_file="auto-mpg.html")

In [53]:
#Dropping car_name
data.drop('car name', axis=1, inplace=True)
data.loc[data.horsepower=='?','horsepower'] = np.nan
data['horsepower'] = data.horsepower.astype(float, errors = 'raise')
data['horsepower']= data['horsepower'].fillna(data['horsepower'].mean())
data.dropna(inplace=True)

## Statsmodels Regression

In [59]:
X = data.drop(['mpg'], axis=1)
y = data['mpg']

In [91]:
## let's add an intercept (beta_0) to our model because Statsmodels does not do that by default
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

/home/manujosephv/anaconda3/envs/blog/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning:

Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.821
Model:                            OLS   Adj. R-squared:                  0.818
Method:                 Least Squares   F-statistic:                     255.4
Date:                Tue, 12 Nov 2019   Prob (F-statistic):          2.25e-141
Time:                        19:34:12   Log-Likelihood:                -1040.3
No. Observations:                 398   AIC:                             2097.
Df Residuals:                     390   BIC:                             2129.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -18.0587      4.583     -3.940      0.000     -27.069      -9.048
cylinders       -0.4183      0.322     -1.299      0.195      -1.051       0.215
displacement     0.0189      0.008      2.518      0.012       0.004       0.034
horsepower      -0.0114      0.013     -0.865      0.388      -0.037       0.015
weight          -0.0067      0.001    -10.480      0.000      -0.008      -0.005
acceleration     0.1026      0.096      1.067      0.287      -0.086       0.292
model year       0.7568      0.050     15.008      0.000       0.658       0.856
origin           1.4175      0.275      5.154      0.000       0.877       1.958
==============================================================================
Omnibus:                       30.551   Durbin-Watson:                   1.289
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               49.230
Skew:                           0.515   Prob(JB):                     2.04e-11
Kurtosis:                       4.381   Cond. No.                     8.49e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.49e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [92]:
model.params[1:].iplot(kind='bar')

In [93]:
#Standardizing the Regression coefficients
std_coeff = model.params
for col in X.columns:
    std_coeff[col] = (std_coeff[col]* np.std(X[col]))/ np.std(y)

In [94]:
std_coeff[1:].iplot(kind='bar')

## Sci-kit Learn

In [109]:
lm = LinearRegression()
lm.fit(X,y)
r2 = lm.score(X,y)
adj_r2 = 1-(1-r2)*(len(X)-1)/(len(X)-len(X.columns)-1)
print ("R2 Score: {:.2f}% | Adj R2 Score: {:.2f}%".format(r2*100,adj_r2*100))
params = pd.Series({'const': lm.intercept_})
for i,col in enumerate(X.columns):
    params[col] = lm.coef_[i]

R2 Score: 82.09% | Adj R2 Score: 81.72%


In [110]:
params[1:].iplot(kind='bar')

**Getting exactly the same results as Stats Models**

In [111]:
from sklearn.preprocessing import StandardScaler

In [122]:
scaler = StandardScaler()
X_std = scaler.fit_transform(X)
lm = LinearRegression()
lm.fit(X_std,y)
r2 = lm.score(X_std,y)
adj_r2 = 1-(1-r2)*(len(X)-1)/(len(X)-len(X.columns)-1)
print ("R2 Score: {:.2f}% | Adj R2 Score: {:.2f}%".format(r2*100,adj_r2*100))
params = pd.Series({'const': lm.intercept_})
for i,col in enumerate(X.columns):
    params[col] = lm.coef_[i]

R2 Score: 82.09% | Adj R2 Score: 81.72%


In [123]:
params[1:].iplot(kind='bar')

## Ridge Regression

In [114]:
lm = RidgeCV()
lm.fit(X,y)
r2 = lm.score(X,y)
adj_r2 = 1-(1-r2)*(len(X)-1)/(len(X)-len(X.columns)-1)
print ("R2 Score: {:.2f}% | Adj R2 Score: {:.2f}%".format(r2*100,adj_r2*100))
params = pd.Series({'const': lm.intercept_})
for i,col in enumerate(X.columns):
    params[col] = lm.coef_[i]

R2 Score: 82.08% | Adj R2 Score: 81.72%


In [115]:
params[1:].iplot(kind='bar')

In [120]:
scaler = StandardScaler()
X_std = scaler.fit_transform(X)
lm = RidgeCV()
lm.fit(X_std,y)
r2 = lm.score(X_std,y)
adj_r2 = 1-(1-r2)*(len(X)-1)/(len(X)-len(X.columns)-1)
print ("R2 Score: {:.2f}% | Adj R2 Score: {:.2f}%".format(r2*100,adj_r2*100))
params = pd.Series({'const': lm.intercept_})
for i,col in enumerate(X.columns):
    params[col] = lm.coef_[i]

R2 Score: 82.08% | Adj R2 Score: 81.72%


In [121]:
params[1:].iplot(kind='bar')